# Empirical deviations of semicircle law in mixed-matrix ensembles

(c) 2021 Mehmet Süzen

This is a supplementary material to the paper with the same title. 

Reproduce the results of generating mixed Gaussian ensembles and 
associated spectral analysis.

See [Full Article](http://dx.doi.org/10.13140/RG.2.2.12015.97443) 


In [ ]:
#
# imports and versions
#
import sys
import numpy as np
import matplotlib
from matplotlib import pyplot as plt

np.__version__, matplotlib.__version__, sys.version


## Core Methods

* `get_goe` : GOE drawn single matrix.
* `get_goe_eigens` : GOE eigenvalues given ensemble size.
* `list2plist` : Convert list to a Cyclic up to higher length list. recall periodic boundary conditions. 
* `get_goe_eigens_periodic` : Generate eigenvalues with PBC GOE.
* `get_goe_eigens_mixed` : Generate eigenvalues with PBC GOE with mixture.
* `spectral_staircase` : Get density-of-states, not needed, retained for reference.

## Generating Gaussian Orthogonal Ensemble (GOE) 

$A = \mathcal{N}(0, 1)$
$GOE(n) = \frac{1}{2}(A + A^{T})$.

$diag(G) = \mathcal{N}(0, 1)$.

$offdiag(G) = \mathcal{N}(0,1/2)$ # symmetric.


In [ ]:
def get_goe(order: int) -> np.array:
    """ Get GOE given order. """
    A = np.zeros(order*order).reshape(order, order)
    tril_ix = np.tril_indices(order, k=-1)
    triu_ix = np.triu_indices(order, k=1)
    diag_ix = np.diag_indices(order)
    off_diagonal = np.random.normal(loc=0, scale=0.5, size=tril_ix[0].shape[0])
    A[tril_ix] = off_diagonal
    A[triu_ix] = off_diagonal
    A[diag_ix] = np.random.normal(loc=0, scale=1, size=diag_ix[0].shape[0])
    return 0.50*(A+A.transpose()) # if not here, eigenvalues will be complex.

def get_goe_eigens(n_ensemble:int, order:int):
    """ Generate eigenvalues of GOE ensemble give ensemble size and order. """
    _eigens = np.empty(0)
    for _ in np.arange(n_ensemble):
        Goe = get_goe(order)
        _eigens = np.append(_eigens, np.linalg.eigvals(Goe))
    return np.real(_eigens)

In [ ]:
%%time
# Unit-test
order = 1000
n_ensemble = 50
eigens = get_goe_eigens(n_ensemble, order)
_ = plt.hist(eigens, bins='auto') # semi-circle law

## Generating Mixed GOE

Given ensemble size and N, produce a mixed ensemble as a list.

In [ ]:
#
# For periodic eigenvalues
# Recall: https://nbviewer.jupyter.org/urls/arxiv.org/src/1911.07831v1/anc/periodic_spectral_ergodicity_dnn.ipynb
 
from itertools import cycle


def list2plist(lst, upper_bound):
    """
    
    Given list lst ans upper_bound.
    Return period_lst, cycle. 
    
    """
    pool = cycle(lst)
    c = 1
    lst_period = []
    for item in pool:
        c = c + 1
        lst_period.append(item)
        if c > upper_bound:
            break
    return lst_period

def get_goe_eigens_periodic(n_ensemble:int, order:int, N_max:int):
    """ Generate eigenvalues of GOE ensemble give ensemble size and order with
        periodic adjustment. """
    eigens = np.empty(0)
    for _ in np.arange(n_ensemble):
        Goe = get_goe(order)
        eigen_ = list2plist(list(np.linalg.eigvals(Goe)), N_max)
        eigens = np.append(eigens, eigen_)
    return eigens

In [ ]:
N = 400  # maximum matrix size ( Bernouilli trials )
p = 0.7  # mixing coefficient ( Probability of success )
M = 1  # ensemble size 

def get_goe_eigens_mixed(N, p, M):
    """ 
    Get eigenvalues of a mixed GOE: list of eigenvalues, count check 
     N # maximum matrix size ( Bernouilli trials )
     p # mixing coefficient ( Probability of success )
     M # ensemble size 
    """
    # number of success over the n trials
    counts_matrix, sizes_matrix = np.histogram(np.random.binomial(N, p, M)) 
    # means how many to generate on which order
    order_count_mapping = [(matrix_order, count_matrix) for matrix_order,count_matrix 
                           in zip(np.int64(np.round(sizes_matrix)), counts_matrix)]
    mixed_ensemble_eigens = np.empty(0)
    size_check = 0
    for order, count in order_count_mapping:
        if count > 0:
            eigens = get_goe_eigens_periodic(count, order, N)
            # with periodic boundary: maximum size of N
            mixed_ensemble_eigens = np.append(mixed_ensemble_eigens, eigens)
        size_check = size_check + order*count
    return mixed_ensemble_eigens, size_check

In [ ]:
counts_matrix, sizes_matrix = np.histogram(np.random.binomial(N, p, M)) 

In [ ]:
counts_matrix, sizes_matrix

In [ ]:
mixed_ensemble_eigens, size_check = get_goe_eigens_mixed(N, p, M)

In [ ]:
size_check, len(mixed_ensemble_eigens)

In [ ]:
len(mixed_ensemble_eigens), size_check

In [ ]:
_ = plt.hist(mixed_ensemble_eigens, bins='auto') # semi-circle law

## Spectral Unfolding   nearest-level-spacing (GOE): 
**we don't need this as eigenvalue drawn from an ensemble not a single matrix**

**retained for reference**

The unfolding procedure is summarized as follows.

1. Compute eigenvalues. $E_{i}$. (coming from ensemble_eigens either normal or mixed and 
   collect all as a single eigenvalue set)
   `get_goe_eigens_mixed` or `get_goe_eigens` 
2. Compute `spectral staircase` $N(E_{i})$ (density of states). This will be already 
   unfolded/trend removed as we used, multiple matrices.
   
3. Detrend $N(E_{i})$ with LOWESS, remove noise component and get 
   $\epsilon_{i} = n(E_{i})$.
   At this point this is an unfolded spectra.   
4. Set Equidistance $x$ on spectra (the difference scale).
5. Cheoen's paper definition of `nearest-level-spacing`: 
   Get pairs $(\epsilon_{j+1}, \epsilon_{j})$ :
   Count the pairs lie in a given spacing and divide 
   with the number of all pairs.mixed_ensemble_eigens

In [ ]:
def spectral_staircase(eigens, number_bin):
    """  Get spectral staircase (density of states) as x, y : min/max as range. """
    max_ = np.round(np.max(eigens), 0) 
    min_ = np.round(np.min(eigens), 0) 
    stairs_ = np.linspace(min_, max_, number_bin)
    dos = [] # density of states
    total_ = len(eigens)
    for stair_ in stairs_:
        dos_stair = len([is_smaller for is_smaller in mixed_ensemble_eigens <= stair_
                    if is_smaller == True])/total_
        dos.append(dos_stair)
    return stairs_, dos

In [ ]:
stairs_, dos = spectral_staircase(mixed_ensemble_eigens, 100)

In [ ]:
plt.plot(stairs_, dos)

In [ ]:
# Level spacing is the number pair differences falling into stairs_

In [ ]:
eigens = mixed_ensemble_eigens

In [ ]:
sorted_eigens = np.sort(eigens)
sorted_eigens_diffs = np.diff(sorted_eigens)

In [ ]:
_ = plt.hist(sorted_eigens_diffs, bins='auto') # semi-circle law

In [ ]:
N = 100
M = 20
p = 0.75
mixed_ensemble_eigens, size_check = get_goe_eigens_mixed(N, p, M)

In [ ]:
eigens = mixed_ensemble_eigens
sorted_eigens = np.sort(eigens)
sorted_eigens_diffs = np.diff(sorted_eigens)
_ = plt.hist(sorted_eigens_diffs, bins='auto') # semi-circle law

In [ ]:
y, x = np.histogram(sorted_eigens_diffs, density=False)
y = y/np.sum(y)

In [ ]:
plt.plot(x[:-1], y)

## Eigenvalues generation for comparing GOE and mixed GOE

Generate data for semi-circle laws and nearest-neighbour spacing for GOE and mixed-GOE

* Matrix sizes N= 400, 1000.
* M = 100 (Fixed ensemble size, number of draws from an ensemble).
* Mixing coefficients p= 0.70, 0.80, 0.95

Data store:  list of dictionaries: 'N', 'p', 'eigens', 'ensemble' (GOE or mGOE).

In [ ]:
%%time
data_mixed =[]
N = 400
p = 0.7
M = 100
ensemble = 'mGOE'
eigens, _ = get_goe_eigens_mixed(N, p, M)
data_mixed.append({'N':N, 'p':p,'eigens':eigens, 'ensemble':ensemble})
p = 0.8
eigens, _ = get_goe_eigens_mixed(N, p, M)
data_mixed.append({'N':N, 'p':p,'eigens':eigens, 'ensemble':ensemble})
p = 0.95
eigens, _ = get_goe_eigens_mixed(N, p, M)
data_mixed.append({'N':N, 'p':p,'eigens':eigens, 'ensemble':ensemble})

In [ ]:
%%time
N = 1000
p = 0.7
M = 100
ensemble = 'mGOE'
eigens, _ = get_goe_eigens_mixed(N, p, M)
data_mixed.append({'N':N, 'p':p,'eigens':eigens, 'ensemble':ensemble})
p = 0.8
eigens, _ = get_goe_eigens_mixed(N, p, M)
data_mixed.append({'N':N, 'p':p,'eigens':eigens, 'ensemble':ensemble})
p = 0.95
eigens, _ = get_goe_eigens_mixed(N, p, M)
data_mixed.append({'N':N, 'p':p,'eigens':eigens, 'ensemble':ensemble})

In [ ]:
%%time
N = 400
M = 100
ensemble = 'GOE'
eigens = get_goe_eigens(M, N)
data_mixed.append({'N':N, 'p': np.nan,'eigens':eigens, 'ensemble':ensemble})
N = 1000
eigens = get_goe_eigens(M, N)
data_mixed.append({'N':N, 'p': np.nan,'eigens':eigens, 'ensemble':ensemble})

## Results Semi-circle law

*  2 plots of Semi-circles for N=400 and N=1000 (each GOE and mGOE with different p)
*  2 plots of nearest neigbour spacing for N=400 and N=1000 (each GOE and mGOE with different p)

In [ ]:
# N = 400
# data_mixed[0], data_mixed[1], data_mixed[2], data_mixed[6]
# N = 1000
# data_mixed[3], data_mixed[4], data_mixed[5], data_mixed[7]

In [ ]:
y0, x0 = np.histogram(data_mixed[0]['eigens'], bins='auto', density=False)
y0 = y0/np.sum(y0)

y1, x1 = np.histogram(data_mixed[1]['eigens'], bins='auto', density=False)
y1 = y1/np.sum(y1)

y2, x2 = np.histogram(data_mixed[2]['eigens'], bins='auto', density=False)
y2 = y2/np.sum(y2)

y3, x3 = np.histogram(data_mixed[6]['eigens'], bins='auto', density=False)
y3 = y3/np.sum(y3)

In [ ]:
font = {"family": "normal", "weight": "bold", "size": 14}

plt.rc("font", **font)

plt.plot(x0[:-1], y0, "--", label="mGOE $\mu$=0.70 ")
plt.plot(x1[:-1], y1, "-*", label="mGOE $\mu$=0.80 ")
plt.plot(x2[:-1], y2, "-", label="mGOE $\mu$=0.95 ")
plt.plot(x3[:-1], y3, "-.", label="GOE ")
plt.ylim([0, 0.09])
plt.legend(loc="upper left")
plt.xlabel("Eigenvalues", **font)
plt.ylabel("Density", **font)
plt.title("Semi-circle law  N=400 ", **font)
plt.savefig("../semicircle400.eps", format="eps", dpi=1000, bbox_inches="tight")
plt.draw()
plt.show()

In [ ]:
y0, x0 = np.histogram(data_mixed[3]['eigens'], bins='auto', density=False)
y0 = y0/np.sum(y0)

y1, x1 = np.histogram(data_mixed[4]['eigens'], bins='auto', density=False)
y1 = y1/np.sum(y1)

y2, x2 = np.histogram(data_mixed[5]['eigens'], bins='auto', density=False)
y2 = y2/np.sum(y2)

y3, x3 = np.histogram(data_mixed[7]['eigens'], bins='auto', density=False)
y3 = y3/np.sum(y3)

In [ ]:
font = {"family": "normal", "weight": "bold", "size": 14}

from matplotlib import pyplot as plt

plt.rc("font", **font)
plt.plot(x0[:-1], y0, "--", label="mGOE $\mu$=0.70 ")
plt.plot(x1[:-1], y1, "-*", label="mGOE $\mu$=0.80 ")
plt.plot(x2[:-1], y2, "-", label="mGOE $\mu$=0.95 ")
plt.plot(x3[:-1], y3, "-.", label="GOE ")
plt.ylim([0, 0.08])
plt.legend(loc="upper center")
plt.xlabel("Eigenvalues", **font)
plt.ylabel("Density", **font)
plt.title("Semi-circle law  N=1000 ", **font)
plt.savefig("../semicircle1000.eps", format="eps", dpi=1000, bbox_inches="tight")
plt.draw()
plt.show()


## Spacing (spectral spacing single )

In [ ]:
%%time
data_mixed =[]
N = 1000
p = 0.7
M = 1
ensemble = 'mGOE'
eigens, _ = get_goe_eigens_mixed(N, p, M)
data_mixed.append({'N':N, 'p':p,'eigens':eigens, 'ensemble':ensemble})
p = 0.8
eigens, _ = get_goe_eigens_mixed(N, p, M)
data_mixed.append({'N':N, 'p':p,'eigens':eigens, 'ensemble':ensemble})
p = 0.95
eigens, _ = get_goe_eigens_mixed(N, p, M)
data_mixed.append({'N':N, 'p':p,'eigens':eigens, 'ensemble':ensemble})
N = 1000
eigens = get_goe_eigens(M, N)
data_mixed.append({'N':N, 'p': np.nan,'eigens':eigens, 'ensemble':ensemble})

In [ ]:
sorted_eigens = np.sort((data_mixed[0]['eigens']))
sorted_eigens_diffs = np.diff(sorted_eigens)
y0, x0 = np.histogram(sorted_eigens_diffs, bins='auto', density=False)
y0 = y0/np.sum(y0)

sorted_eigens = np.sort((data_mixed[1]['eigens']))
sorted_eigens_diffs = np.diff(sorted_eigens)
y1, x1 = np.histogram(sorted_eigens_diffs, bins='auto', density=False)
y1 = y1/np.sum(y1)

sorted_eigens = np.sort((data_mixed[2]['eigens']))
sorted_eigens_diffs = np.diff(sorted_eigens)
y2, x2 = np.histogram(sorted_eigens_diffs, bins='auto', density=False)
y2 = y2/np.sum(y2)

sorted_eigens = np.sort((data_mixed[3]['eigens']))
sorted_eigens_diffs = np.diff(sorted_eigens)
y3, x3 = np.histogram(sorted_eigens_diffs, bins='auto', density=False)
y3 = y3/np.sum(y3)

In [ ]:
font = {"family": "normal", "weight": "bold", "size": 14}

from matplotlib import pyplot as plt

plt.rc("font", **font)

plt.plot(x0[:-1], y0, "--", label="mGOE $\mu$=0.70 ")
plt.plot(x1[:-1], y1, "-*", label="mGOE $\mu$=0.80 ")
plt.plot(x2[:-1], y2, "-", label="mGOE $\mu$=0.95 ")
plt.plot(x3[:-1], y3, "-.", label="GOE ")
plt.xlim([0, 0.2])
plt.ylim([0, 0.2])
plt.legend(loc="upper center")
plt.xlabel("Eigenvalue Spacing", **font)
plt.ylabel("Density", **font)
plt.title("Nearest-neighbour spacing N=1000 ", **font)
plt.savefig("../neighbour1000.eps", format="eps", dpi=1000, bbox_inches="tight")
plt.draw()
plt.show()